 # **DASHBOARD DEPLOYMENT CODES**

 The dashboard code was run with spider using flask the justification for subbmiting with google colab was because of the size of the saved model which was tool large to be uploaded on github

SVM SAVED MODEL LINK

https://drive.google.com/file/d/13LtfGeXYWTq5zDMcARI_uf0kZe5qTHnH/view?usp=sharing

APPLICATION

In [ ]:
from flask import Flask, request, jsonify, render_template, send_from_directory
import numpy as np
import os
import pickle
from PIL import Image
import io
import uuid

# Initialize Flask app
app = Flask(__name__)

# Directory to store uploaded images
UPLOAD_FOLDER = 'uploads'
if not os.path.exists(UPLOAD_FOLDER):
    os.makedirs(UPLOAD_FOLDER)

# Load the trained model
model_filename = "C:\\Users\\olaiy\\Desktop\\deploymet2nicedashboad\\svm_model_for_deployment.pkl"
with open(model_filename, 'rb') as file:
    data = pickle.load(file)
    model = data['svm_model']

# Define the image preprocessing function
def preprocess_image(image_bytes):
    image = Image.open(io.BytesIO(image_bytes)).convert('L')  # Convert to grayscale
    image = image.resize((256, 256))  # Resize to the fixed size used during training
    image_array = np.array(image).flatten()  # Flatten the image
    return image_array.reshape(1, -1)  # Reshape for prediction

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/classify')
def classify():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'}), 400

    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'No selected file'}), 400

    if file and file.filename.lower().endswith(('jpeg', 'jpg', 'png')):
        try:
            # Save the uploaded image
            filename = str(uuid.uuid4()) + os.path.splitext(file.filename)[-1]
            filepath = os.path.join(UPLOAD_FOLDER, filename)
            file.save(filepath)

            # Read and preprocess the image
            with open(filepath, 'rb') as image_file:
                image_bytes = image_file.read()
            image_array = preprocess_image(image_bytes)

            # Make prediction
            prediction = model.predict(image_array)[0]

            # Return the prediction and the image path
            return jsonify({
                'prediction': prediction,
                'image_url': f'/uploads/{filename}'
            })
        except Exception as e:
            return jsonify({'error': str(e)}), 500
    else:
        return jsonify({'error': 'Invalid file type'}), 400

@app.route('/uploads/<filename>')
def uploaded_file(filename):
    return send_from_directory(UPLOAD_FOLDER, filename)

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=True)


**INDEX**

In [ ]:
<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Alzheimer's Disease Dashboard</title>
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.0.0-beta3/css/all.min.css"> <!-- Font Awesome CDN -->
    <style>
        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            margin: 0;
            padding: 0;
            background: url('https://source.unsplash.com/random/1920x1080/?purple,abstract') no-repeat center center fixed;
            background-size: cover;
            color: #f0e6f6;
        }

        header {
            background: linear-gradient(90deg, rgba(93, 12, 255, 0.7), rgba(155, 89, 182, 0.7)); /* Purple gradient */
            color: #f0e6f6;
            padding: 20px;
            text-align: center;
            box-shadow: 0 8px 16px rgba(0, 0, 0, 0.3);
            border-bottom: 3px solid #8e44ad; /* Border to add definition */
        }

        header h1 {
            margin: 0;
            font-size: 2.8em;
            letter-spacing: 1px;
        }

        .container {
            display: none;
            padding: 40px;
            text-align: center;
            background-color: rgba(142, 68, 173, 0.8); /* Deeper purple for all pages */
            border-radius: 15px;
            box-shadow: 0 8px 16px rgba(0, 0, 0, 0.3);
            margin: 20px auto;
            max-width: 800px;
        }

        .active {
            display: block;
        }

        .btn {
            padding: 14px 28px;
            font-size: 18px;
            color: #f0e6f6;
            background: linear-gradient(90deg, #8e44ad, #9b59b6); /* Gradient purple button */
            border: none;
            border-radius: 30px;
            cursor: pointer;
            margin: 20px;
            transition: background 0.3s, transform 0.3s;
        }

        .btn:hover {
            background: linear-gradient(90deg, #732d91, #8e44ad); /* Darker gradient on hover */
            transform: translateY(-3px);
            box-shadow: 0 8px 16px rgba(0, 0, 0, 0.2);
        }

        input[type="file"] {
            margin: 20px 0;
            padding: 12px;
            font-size: 16px;
            color: #f0e6f6;
            background-color: #8e44ad; /* Purple background */
            border: none;
            border-radius: 8px;
            cursor: pointer;
            box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
            transition: box-shadow 0.3s;
        }

        input[type="file"]:hover {
            box-shadow: 0 8px 16px rgba(0, 0, 0, 0.2);
        }

        .image-preview {
            margin-top: 20px;
            border-radius: 15px;
            box-shadow: 0 8px 16px rgba(0, 0, 0, 0.3);
        }

        #result {
            margin-top: 30px;
            font-size: 22px;
            font-weight: bold;
        }

        .icon {
            color: #f0e6f6; /* White color for icons */
            font-size: 24px; /* Adjust size as needed */
        }

        .icon:hover {
            color: #9b59b6; /* Light purple color on hover */
        }
    </style>
</head>

<body>
    <header>
        <h1>Alzheimer's Disease Detection Dashboard</h1>
    </header>

    <!-- Introduction Page -->
    <div id="introPage" class="container intro active">
        <h2>Welcome to the Alzheimer's Disease Detection Dashboard</h2>
        <p>Our tool helps you analyse and classify images related to Alzheimer's disease. By employing machine learning, we provide predictions to aid in early detection and research. You can upload an image to see its classification result.</p>
        <button class="btn" onclick="showPage('classifyPage')">Go to Image Classifier</button>
    </div>

    <!-- Classification Page -->
    <div id="classifyPage" class="container content">
        <h2>Image Detection</h2>
        <form id="uploadForm" enctype="multipart/form-data">
            <input type="file" id="fileInput" name="file" accept="image/jpeg, image/png" required>
            <button type="button" class="btn" onclick="uploadImage()">Upload and Predict</button>
        </form>
        <div id="result"></div>
        <img id="imagePreview" class="image-preview" style="display:none; max-width: 100%; max-height: 400px;">
        <button class="btn" onclick="showPage('introPage')">Back to Introduction</button>
    </div>

    <!-- Icons example -->
    <div style="text-align: center; margin: 20px;">
        <i class="fas fa-home icon"></i>
        <i class="fas fa-upload icon" style="margin-left: 20px;"></i>
        <i class="fas fa-info-circle icon" style="margin-left: 20px;"></i>
    </div>

    <script>
        function showPage(pageId) {
            document.querySelectorAll('.container').forEach(container => {
                container.classList.remove('active');
            });
            document.getElementById(pageId).classList.add('active');
        }

        function uploadImage() {
            const formData = new FormData(document.getElementById('uploadForm'));

            fetch('/predict', {
                method: 'POST',
                body: formData
            })
            .then(response => response.json())
            .then(data => {
                if (data.error) {
                    document.getElementById('result').textContent = `Error: ${data.error}`;
                    document.getElementById('imagePreview').style.display = 'none';
                } else {
                    document.getElementById('result').textContent = `Prediction: ${data.prediction}`;
                    document.getElementById('imagePreview').src = data.image_url;
                    document.getElementById('imagePreview').style.display = 'block';
                }
            })
            .catch(error => {
                document.getElementById('result').textContent = `Error: ${error}`;
                document.getElementById('imagePreview').style.display = 'none';
            });
        }
    </script>
</body>

</html>
